# SAP Cloud ALM Project and Task API Test and Demo
## Adding and Removing Users from Project Teams

This notebook contains examples of SAP Business Hub API calls for SAP Cloud ALM for Implementation. The specific APIs shown below are for Projects and Tasks.

The API information and specification is available here:

* https://api.sap.com/package/SAPCloudALM/rest - SAP Cloud ALM
* https://api.sap.com/api/CALM_PJM/overview - SAP Cloud ALM Projects
* https://api.sap.com/api/CALM_TKM/overview - SAP Cloud ALM Tasks

Please note the license and other terms and conditions contained in this notebook's repository: https://github.com/SAP-samples/cloud-alm-api-examples

## Python Dependencies Required

In order to run the samples in this notebook, install the following dependencies:

* Jupyter integration in Visual Studio Code: https://code.visualstudio.com/docs/python/jupyter-support 
* Python 3, a recent version, is Required. Python 3.8 was used here
* Requests - for handling HTTP GET/POST/PATCH/DELETE Requests - https://docs.python-requests.org/en/latest/user/install/#install
* Requests-OAuthlib - for authentication with requests - https://requests-oauthlib.readthedocs.io/en/latest/index.html#installation
* Pandas - Python data analysis - https://pandas.pydata.org/docs/getting_started/install.html

## APIs called

API to get projects: https://<tenant url\>/api/imp-pjm-srv/v1/projects

API to get tasks: https://<tenant url\>/api/imp-tkm-srv/v1/tasks?projectId=<project ID\>

## Updates

* 2025-07 Update users assigned to a project.
* API specification: <https://api.sap.com/api/CALM_PJM/path/patch_teams__teamId__roles__roleId_>


---

## Authentication information

You must create a python module file called `apidata.py` and put the information specific to your tenant there. This inclduds

* OAuth2 client ID and secret
* Token url
* Base URL for API calls

Get client ID and secret variables from an external module: this information is senstive.

These items can be retrieved from the BTP Cockpit 

### Format of module apidata.py for import

```python
ptm_all_client_id = r'get your client ID from BTP Cockpit'
ptm_all_client_secret = r'get your client secret from BTP Cockpit'
token_url = 'your token url'
base_url = 'your base url'
```


In [ ]:
import apidata as ad

client_id = ad.ptm_all_client_id
client_secret = ad.ptm_all_client_secret
token_url = ad.token_url
base_url = ad.base_url

### Get token for authentication

Call OAuth token API with credential information. Add the resulting header to all requests.

See Requests-OAuthlib documentation for Backend Application Flow:

* https://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html#backend-application-flow

In [ ]:
import requests
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient

client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url=token_url, client_id=client_id,
        client_secret=client_secret)

hed = {'Authorization': 'Bearer ' + token['access_token']}        


---

## Perform GET request to retrieve list of all projects

Expected response: "200 OK"

In [ ]:
response = requests.get(base_url + '/api/calm-projects/v1/projects', headers=hed)

print(response.status_code, response.reason)

## Parse JSON into Pandas Dataframe

This takes the data returned from the projects API, which is in JSON format, and places it into a dataframe for further processing and analysis.

In [ ]:
import pandas as pd

df = pd.json_normalize(response.json())

df

## Get a Project Name 

You may wish to adjust the code below to set a project name of your choosing, for example 'My API Test Project'. In this sample, we'll just take the first project in the list.


In [ ]:
sample_project_name = df.iloc[0]['name'] # First project in the list
sample_project_name

### Override Sample Project Name for Demo

I'm setting my own project name here, because I have one ready for my demos. You can do the same if you don't want to use the first project.

In [ ]:

sample_project_name = 'My API Test Project'


### Get ID of a Selected Project

Read ID of project `sample_project_name` from dataframe

In [ ]:
staffingPrjID = df.loc[df['name'] == sample_project_name]['id'].values[0]

print('Project:', sample_project_name, 'ID:', staffingPrjID)

---
## 2025-07-09 - Update Users in a Project

API path to PATCH with users:

`/teams/{teamId}/roles/{roleId}`

See the Open API specification for the PATCH operation: <https://api.sap.com/api/CALM_PJM/path/patch_teams__teamId__roles__roleId_>

In [ ]:
# Get the teams in the project:

staffingPath = f'{base_url}/api/calm-projects/v1/projects/{staffingPrjID}/teams'

print(staffingPath)

response = requests.get(staffingPath, headers=hed)

print(response.status_code, response.reason)
print(response.json())

In [ ]:
# Get the roles in the first team:

firstTeamId = response.json()[0]['id']

teamRolesPath = f'{base_url}/api/calm-projects/v1/teams/{firstTeamId}/roles'

print(teamRolesPath)

response = requests.get(teamRolesPath, headers=hed)

print(response.status_code, response.reason)

In [ ]:
# Read the roles into a dataframe for easy processing:
dfRoles = pd.json_normalize(response.json())

dfRoles

In [ ]:
# Get the ID of the "Configuration Expert" role:
configExpertRole = dfRoles.loc[dfRoles['roleName'] == 'Configuration Expert']
configExpertRoleID = configExpertRole['roleId'].values[0] 
configExpertRoleID

In [ ]:
configExpertRole

### Modify the role setting the "members" array

We need the user's e-mail address. Provide this in the `members` array.

The contents of the array **replaces** the existing users assigned to the role. If you are just adding a user, read the original array and append the new users to it before your send the PATCH request.

In [ ]:
roleToModify = {
  "members": [
    {
      "userEmail": "test@example.com"
    },
    {
      "userEmail": "test.2@example.com"
    }
  ]
}

roleToModify

In [ ]:
# Send the PATCH request:

configExpertTeamRolesPath = f'{base_url}/api/calm-projects/v1/teams/{firstTeamId}/roles/{configExpertRoleID}'

print(configExpertTeamRolesPath)

response = requests.patch(configExpertTeamRolesPath, headers=hed, json=roleToModify)

print(response.status_code, response.reason)


### Check the users returned from the request

If the request returns `_FORMER_MEMBER_` instead of the user ID, this means that the user does not exist in the tenant. If the system is appropriately configured, it creates an approval request in the User Management application.

In [ ]:
if response.status_code == 200:
    print(response.json()['members'])